To compare the means of consecutive yearly blocks in our solar energy dataset, we should use a **paired t-test**. Here's why:

### Paired t-test Explanation
- **Paired t-test**: This test is used when you have two related groups and you want to determine if there is a statistically significant difference between their means. The test assumes that the pairs are dependent, which fits our scenario since we are comparing consecutive time blocks (e.g., 1980-1983 vs. 1984-1987) that are inherently related (because they are successive periods).

### Hypothesis for Our Test
Given our goal, the hypotheses for each comparison would be:

- **Null Hypothesis (H₀)**: There is no decrease in mean solar energy generation between the two blocks (mean of block 1 ≤ mean of block 2).
- **Alternative Hypothesis (H₁)**: There is a decrease in mean solar energy generation between the two blocks (mean of block 1 > mean of block 2).

### Steps to Perform the Paired t-test
1. **Group our data** into consecutive 4-year blocks.
2. **Calculate the mean** of solar energy generation for each block.
3. **Perform the paired t-test** on the means of consecutive blocks.
4. Repeat the test for all consecutive block pairs (e.g., 1980-1983 vs. 1984-1987, 1984-1987 vs. 1988-1991, etc.).

### Test Direction
Since we are specifically interested in whether the mean has **decreased**, we would use a **one-tailed paired t-test**.

### Example
For instance, if `X₁` represents the mean for 1980-1983 and `X₂` for 1984-1987, we would check if \( X₁ > X₂ \).

This approach will allow us to determine if there's a statistically significant decrease in mean solar energy generation from one block to the next over the 44-year period.

If the p-value is less than the significance level (e.g., 0.05), we would reject the null hypothesis and conclude that there is a significant decrease in mean solar energy generation between the two blocks. Otherwise, we would fail to reject the null hypothesis.

In [1]:
import pandas as pd
import numpy as np

data_jakarta = pd.read_csv('1980-2023 renewable energy data/ninja_pv_-7.2623_112.7361_1980.csv', header=3)
name = "1980-2023 renewable energy data/ninja_pv_-7.2623_112.7361_"

for i in range(1981, 2024):
    data_jakarta = pd.concat([data_jakarta, pd.read_csv(name + str(i) + '.csv', header=3)])

data_jakarta.drop(columns=['time'], inplace=True)
data_jakarta['local_time'] = pd.to_datetime(data_jakarta['local_time'])

data_jakarta['local_time'] = data_jakarta['local_time'].dt.year
data_jakarta = data_jakarta.groupby('local_time').sum()
#data_jakarta.reset_index(inplace=True)
data_jakarta = data_jakarta.iloc[:-1,:]

data_jakarta.tail()

,electricity
local_time,
2019,1949969.043
2020,1754165.795
2021,1646240.327
2022,1513700.956
2023,1846196.197


In [2]:
def onetailed_paired_ttest(data1, data2):
    from scipy.stats import ttest_rel
    t_stat, p_value = ttest_rel(data1, data2, alternative='greater')
    return t_stat, p_value

## **4Y Block Pairings**

In [3]:
data_4Y = {}

for i in range(1980, 2024, 4):
    data_4Y[f'{i}-{i+3}'] = data_jakarta.loc[i:i+3, 'electricity'].values.tolist()

data_4Y

{'1980-1983': [1984215.04, 1838153.82, 2068803.876, 1926352.241],
 '1984-1987': [1885275.517, 1940873.569, 1951143.692, 2105383.706],
 '1988-1991': [1923381.114, 1799819.844, 1942424.567, 2001966.894],
 '1992-1995': [1777557.862, 1970400.155, 1990792.359, 1790723.418],
 '1996-1999': [1862215.666, 1973296.457, 1633907.377, 1713535.308],
 '2000-2003': [1683637.982, 1783958.153, 1925298.682, 1883313.057],
 '2004-2007': [1900290.624, 1826529.309, 1842572.17, 1786635.778],
 '2008-2011': [1790393.904, 1826130.974, 1441530.109, 1763011.252],
 '2012-2015': [1815850.831, 1641965.98, 1781967.197, 1895202.71],
 '2016-2019': [1601410.129, 1700190.457, 1813670.267, 1949969.043],
 '2020-2023': [1754165.795, 1646240.327, 1513700.956, 1846196.197]}

In [4]:
table_4Y = {
    'Block1': [],
    'Block2': [],
    'B1_mean': [],
    'B2_mean': [],
    'B2-B1': [],
    't_stat': [],
    'p_value': [],
    'Reject H0': []
}

for i in range(0, 10):
    block1 = data_4Y[f'{1980 + i * 4}-{1983 + i * 4}']
    block2 = data_4Y[f'{1984 + i * 4}-{1987 + i * 4}']
    t_stat, p_value = onetailed_paired_ttest(block1, block2)
    reject_H0 = p_value < 0.05
    table_4Y['Block1'].append(f'{1980 + i * 4}-{1983 + i * 4}')
    table_4Y['Block2'].append(f'{1984 + i * 4}-{1987 + i * 4}')
    table_4Y['B1_mean'].append(np.mean(block1))
    table_4Y['B2_mean'].append(np.mean(block2))
    table_4Y['B2-B1'].append(np.mean(block2) - np.mean(block1))
    table_4Y['t_stat'].append(t_stat)
    table_4Y['p_value'].append(p_value)
    table_4Y['Reject H0'].append(reject_H0)

table_4Y = pd.DataFrame(table_4Y)
table_4Y

,Block1,Block2,B1_mean,B2_mean,B2-B1,t_stat,p_value,Reject H0
0,1980-1983,1984-1987,1.954381e+06,1.970669e+06,16287.87675,-0.221011,0.580364,False
1,1984-1987,1988-1991,1.970669e+06,1.916898e+06,-53771.01625,1.299245,0.142347,False
2,1988-1991,1992-1995,1.916898e+06,1.882368e+06,-34529.65625,0.393170,0.360234,False
3,1992-1995,1996-1999,1.882368e+06,1.795739e+06,-86629.74650,0.902845,0.216578,False
4,1996-1999,2000-2003,1.795739e+06,1.819052e+06,23313.26650,-0.190724,0.569541,False
5,2000-2003,2004-2007,1.819052e+06,1.839007e+06,19955.00175,-0.274648,0.599293,False
6,2004-2007,2008-2011,1.839007e+06,1.705267e+06,-133740.41050,1.451157,0.121320,False
7,2008-2011,2012-2015,1.705267e+06,1.783747e+06,78480.11975,-0.718169,0.737735,False
8,2012-2015,2016-2019,1.783747e+06,1.766310e+06,-17436.70550,0.264468,0.404274,False
9,2016-2019,2020-2023,1.766310e+06,1.690076e+06,-76234.15525,0.819877,0.236180,False


In [5]:
table_4Y.to_excel('results/t-tests/ttest-results.xlsx', sheet_name='4Y Blocks', index=False)

## **11Y Block Pairings**

In [6]:
data_11Y = {}

for i in range(1980, 2024, 11):
    data_11Y[f'{i}-{i+10}'] = data_jakarta.loc[i:i+10, 'electricity'].values.tolist()

data_11Y

{'1980-1990': [1984215.04,
  1838153.82,
  2068803.876,
  1926352.241,
  1885275.517,
  1940873.569,
  1951143.692,
  2105383.706,
  1923381.114,
  1799819.844,
  1942424.567],
 '1991-2001': [2001966.894,
  1777557.862,
  1970400.155,
  1990792.359,
  1790723.418,
  1862215.666,
  1973296.457,
  1633907.377,
  1713535.308,
  1683637.982,
  1783958.153],
 '2002-2012': [1925298.682,
  1883313.057,
  1900290.624,
  1826529.309,
  1842572.17,
  1786635.778,
  1790393.904,
  1826130.974,
  1441530.109,
  1763011.252,
  1815850.831],
 '2013-2023': [1641965.98,
  1781967.197,
  1895202.71,
  1601410.129,
  1700190.457,
  1813670.267,
  1949969.043,
  1754165.795,
  1646240.327,
  1513700.956,
  1846196.197]}

In [7]:
table_11Y = {
    'Block1': [],
    'Block2': [],
    'B1_mean': [],
    'B2_mean': [],
    'B2-B1': [],
    't_stat': [],
    'p_value': [],
    'Reject H0': []
}

for i in range(0, 3):
    block1 = data_11Y[f'{1980 + i * 11}-{1990 + i * 11}']
    block2 = data_11Y[f'{1991 + i * 11}-{2001 + i * 11}']
    t_stat, p_value = onetailed_paired_ttest(block1, block2)
    reject_H0 = p_value < 0.05
    table_11Y['Block1'].append(f'{1980 + i * 11}-{1990 + i * 11}')
    table_11Y['Block2'].append(f'{1991 + i * 11}-{2001 + i * 11}')
    table_11Y['B1_mean'].append(np.mean(block1))
    table_11Y['B2_mean'].append(np.mean(block2))
    table_11Y['B2-B1'].append(np.mean(block2) - np.mean(block1))
    table_11Y['t_stat'].append(t_stat)
    table_11Y['p_value'].append(p_value)
    table_11Y['Reject H0'].append(reject_H0)

table_11Y = pd.DataFrame(table_11Y)
table_11Y

,Block1,Block2,B1_mean,B2_mean,B2-B1,t_stat,p_value,Reject H0
0,1980-1990,1991-2001,1.942348e+06,1.834727e+06,-107621.395909,2.457451,0.016912,True
1,1991-2001,2002-2012,1.834727e+06,1.800142e+06,-34584.994636,0.819512,0.215796,False
2,2002-2012,2013-2023,1.800142e+06,1.740425e+06,-59716.148364,1.231478,0.123159,False


In [8]:
with pd.ExcelWriter('results/t-tests/ttest-results.xlsx', mode='a') as writer:
    table_11Y.to_excel(writer, sheet_name='11Y Blocks', index=False)

In [9]:
# Store the 4Y and 11Y blocks data in two separate sheets
with pd.ExcelWriter('results/t-tests/ttest-results.xlsx', mode='a') as writer:
    pd.DataFrame(data_4Y).to_excel(writer, sheet_name='4Y Blocks Data', index=False)
    pd.DataFrame(data_11Y).to_excel(writer, sheet_name='11Y Blocks Data', index=False)